##### Wiktoria Szymańska 140790

##### Laboratorium KCK

### **Projekt**: Rozpoznawanie płci po głosie.


Celem projektu jest stworzenie prostego systemu do rozpoznawania płci na podstawie próbek dźwiękowych głosu. System wykorzystuje analizę częstotliwości fundamentalnych głosu, aby określić, czy głos należy do mężczyzny czy kobiety.


Dane wejściowe:

Krótkie nagrania w formacie WAV. Do celów testowych przygotowano niewielki zbiór nagrań, które umieszczono w folderze *'input_voice_recognition'*. Zawiera on po 7 nagrań z głosami męskimi i żeńskimi umieszczonymi w odpowiednich podkatalogach: *'male'* i *'female'*.

Źródło danych do testowania:

https://commons.wikimedia.org/wiki/Category:Audio_files_of_males_speaking_English

https://commons.wikimedia.org/wiki/Category:Audio_files_of_females_speaking_English


Implementacja:

Funkcja *analyze_voice* analizuje dźwiękowe pliki WAV, wyznaczając częstotliwości fundamentalne głosu. Wykorzystuje do tego operację FFT (Fast Fourier Transform) w celu przekształcenia sygnału dźwiękowego z dziedziny czasu do dziedziny częstotliwości.

Na podstawie średniej częstotliwości fundamentalnej obliczonej dla danej próbki dźwiękowej, program decyduje, czy głos należy do mężczyzny czy kobiety. Wartość 165 Hz jest używana jako próg decyzyjny - głosy o średniej częstotliwości poniżej tego progu są rozpoznawane jako męskie, a powyżej jako żeńskie.

In [135]:
import os
import numpy as np
import wave

In [136]:
def analyze_voice(filename):
    with wave.open(filename, 'rb') as wav:
        frame_rate = wav.getframerate()
        n_frames = wav.getnframes()
        n_channels = wav.getnchannels()
        sample_width = wav.getsampwidth()

        frames = wav.readframes(n_frames)

        if sample_width == 1:
            dtype = np.int8
        elif sample_width == 2:
            dtype = np.int16
        else:
            print("Unsupported sample width.")
            return None

        # decode frames to samples (multiple channels)
        samples = np.frombuffer(frames, dtype=dtype)
        samples = samples.reshape(-1, n_channels)

        # FFT
        spectrum = np.fft.fft(samples, axis=0)
        power_spectrum = np.abs(spectrum) ** 2

        # frequencies corresponding to each sample in the spectrum
        frequencies = np.fft.fftfreq(len(power_spectrum), d=1/frame_rate)

        # peak frequency for each channel (within range 50 - 300 Hz)
        min_freq_index = np.argmax(frequencies > 50)
        max_freq_index = np.argmax(frequencies > 300)
        max_power_indices = np.argmax(power_spectrum[min_freq_index:max_freq_index], axis=0)
        fundamental_frequencies = frequencies[min_freq_index:max_freq_index][max_power_indices]

        # determine gender
        mean_fundamental_frequency = np.mean(fundamental_frequencies)
        gender = 'male' if mean_fundamental_frequency < 165 else 'female'

        return gender


In [137]:
def process_folder(folder_path):
    correct_count = 0
    total_count = 0
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(subdir, file)
                # Prawdziwa płeć na podstawie nazwy podkatalogu
                true_gender = os.path.basename(subdir)
                gender = analyze_voice(file_path)
                correct = 'correctly' if true_gender == gender else 'incorrectly'
                if correct == 'correctly':
                    correct_count += 1
                total_count += 1

                # kolor czerwony lub zielony
                color = '\033[92m' if correct == 'correctly' else '\033[91m'
                print("File: {}\t Gender: {}\t Recognized: {}{}{}".format(file, gender, color, correct, '\033[0m'))

    accuracy = (correct_count / total_count) * 100
    print("Accuracy: {:.2f}%".format(accuracy))


input_folder = 'input_voice_recognition'
process_folder(input_folder)


File: Ann_Daniels_voice.wav	 Gender: female	 Recognized: correctly
File: Mary_Mackey_voice.wav	 Gender: female	 Recognized: correctly
File: Joy_Buolamwini_voice.wav	 Gender: female	 Recognized: correctly
File: Emma_Freud_voice.wav	 Gender: female	 Recognized: correctly
File: Lady_Cobham_voice.wav	 Gender: male	 Recognized: incorrectly
File: Heather_Ford_voice.wav	 Gender: female	 Recognized: correctly
File: Sarah_Outen_voice.wav	 Gender: female	 Recognized: correctly
File: sample.wav	 Gender: male	 Recognized: correctly
File: Antony_John_Williams_voice.wav	 Gender: female	 Recognized: incorrectly
File: Keri_Davies_voice.wav	 Gender: male	 Recognized: correctly
File: Michael_Sheen_voice.wav	 Gender: male	 Recognized: correctly
File: Brian_Dunning_voice.wav	 Gender: male	 Recognized: correctly
File: Dougie_Brown_voice.wav	 Gender: female	 Recognized: incorrectly
File: Chris_Woakes_voice.wav	 Gender: male	 Recognized: correctly
File: Piers_Gibbon_voice.wav	 Gender: male	 Recognized: corre

Wyniki:

W oparciu o zestaw testowy danych dźwiękowych uzyskano dokładność rozpoznania płci na poziomie 80%.
Metoda oparta na FFT jest prosta w implementacji, ale nie gwarantuje pełnej dokładności w rozpoznawaniu płci, co wynika z braku uwzględnienia innych istotnych cech głosu oraz indywidualnych różnic między poszczególnymi głosami.

In [138]:
print(analyze_voice('input_voice_recognition/male/sample.wav'))

male
